<a href="https://colab.research.google.com/github/dr-song-summer-project/AI/blob/main/AIHub_%EA%B0%90%EC%84%B1%EB%A7%90%EB%AD%89%EC%B9%98_%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch

     |████████████████████████████████| 46.9 MB 40 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 4.1 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595744 sha256=166291c92db6138f755f6919955996b9b61db8c2f94028c3aa0147ad3f1885bb
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 4.3 MB/s 
     |████████████████████████████████| 754 kB 3.9 MB/s 
     |████████████████████████████████| 3.0 MB 16.4 MB/s 
     |████████████████████████████████| 895 kB 54.5 MB/s 


In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-7ziaeabn
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-7ziaeabn
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12771 sha256=5af64b0883351079bf5a15bf66d3cfc3344ab9cc008e5ad0d99f11d6f67221e6
  Stored in directory: /tmp/pip-ephem-wheel-cache-w7zzgivq/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [ ]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
##GPU 사용 시
device = torch.device("cuda:0")

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


데이터 입력부  #학습용 데이터셋 불러오기


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 학습용 데이터셋 불러오기
import pandas as pd
dataset_train1 = pd.read_excel('/content/drive/My Drive/닥터송 여름 프로젝트/4. 대-스타 해결 2/감성대화/Training/training_data.xlsx')
dataset_train1.head()

,id,문장,감정
0,44164,아내가 드디어 출산하게 되어서 정말 신이 나.,기쁨
1,3926,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,불안
2,50882,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,당황
3,31303,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,기쁨
4,47200,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,기쁨


 #데이터 전처리

In [ ]:
# 데이터 전처리
# dataset_train1.drop(['id', '연령', '성별', '상황키워드', '신체질환', '감정_소분류', '시스템응답1', '사람문장2', '시스템응답2', '사람문장3', '시스템응답3', '사람문장4', '시스템응답4'], axis=1, inplace=True)
dataset_train1.drop(['id'], axis=1, inplace=True)
dataset_train1.head()

dataset_train1['감정'].unique()

array(['기쁨', '불안', '당황', '슬픔', '분노', '상처'], dtype=object)

In [ ]:
# 대표 질병 5개 추출
data1 = dataset_train1[dataset_train1['감정'] == '기쁨']
data2 = dataset_train1[dataset_train1['감정'] == '불안']
data3 = dataset_train1[dataset_train1['감정'] == '당황']
data4 = dataset_train1[dataset_train1['감정'] == '슬픔']
data5 = dataset_train1[dataset_train1['감정'] == '분노']
data6 = dataset_train1[dataset_train1['감정'] == '상처']
new_data = data1.append([data2, data3, data4, data5, data6], sort=False)
new_data = pd.DataFrame(new_data)
new_data.head()

,문장,감정
0,아내가 드디어 출산하게 되어서 정말 신이 나.,기쁨
3,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,기쁨
4,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,기쁨
14,남편이 죽기 전에 손자들 얼굴 보고 싶다고 했는데 다행히 늦지 않아서 안도했어.,기쁨
23,친구가 사준 로또가 당첨이 됐어.,기쁨


In [ ]:
# 질병명 라벨링
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(new_data['감정'])
new_data['감정'] = encoder.transform(new_data['감정'])

new_data.head()

,문장,감정
0,아내가 드디어 출산하게 되어서 정말 신이 나.,0
3,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,0
4,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,0
14,남편이 죽기 전에 손자들 얼굴 보고 싶다고 했는데 다행히 늦지 않아서 안도했어.,0
23,친구가 사준 로또가 당첨이 됐어.,0


In [ ]:
# 라벨링된 질병명 매핑 ex) {0: '기쁨', 1: '불안', 2: '당황', 3: '슬픔', 4: '분노'}
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
mapping

{0: '기쁨', 1: '당황', 2: '분노', 3: '불안', 4: '상처', 5: '슬픔'}

In [ ]:
# Train / Test set 분리
from sklearn.model_selection import train_test_split
train, test = train_test_split(new_data, test_size=0.2, random_state=42)
print("train shape is:", len(train))
print("test shape is:", len(test))

# 기본 Bert tokenizer 사용
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

train shape is: 32703
test shape is: 8176
using cached model


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset.values] 
        self.labels = [np.int32(i[label_idx]) for i in dataset.values]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
## Setting parameters
max_len = 64 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [ ]:
data_train = BERTDataset(train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(test, 0, 1, tok, max_len, True, False)

In [ ]:
# 데이터 형식 확인 
data_train[0]

(array([   2, 4627, 5330, 3842, 7864, 6855,  517,   54, 4626, 7828, 4627,
        6916, 6553, 1698, 2948, 7706,  517,   54,    3,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(19, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 5)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                #  num_classes=2, #이중분류
                 num_classes=6, #softmax 사용 (다중분류)
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

NameError: ignored

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/2344 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 0.6871083378791809 train acc 0.578125
epoch 1 batch id 201 loss 0.5162928700447083 train acc 0.568407960199005
epoch 1 batch id 401 loss 0.5160441994667053 train acc 0.6689136533665836
epoch 1 batch id 601 loss 0.44791677594184875 train acc 0.7204658901830283
epoch 1 batch id 801 loss 0.4326016902923584 train acc 0.7501365480649188
epoch 1 batch id 1001 loss 0.335864394903183 train acc 0.7684971278721279
epoch 1 batch id 1201 loss 0.35796475410461426 train acc 0.7825509991673605
epoch 1 batch id 1401 loss 0.35910698771476746 train acc 0.792848857958601
epoch 1 batch id 1601 loss 0.3227017819881439 train acc 0.8016474078700812
epoch 1 batch id 1801 loss 0.28074708580970764 train acc 0.8087347307051638
epoch 1 batch id 2001 loss 0.3710687458515167 train acc 0.8150924537731135
epoch 1 batch id 2201 loss 0.27455267310142517 train acc 0.8202663562017265
epoch 1 train acc 0.8242431918373152


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/782 [00:00<?, ?it/s]

epoch 1 test acc 0.8821131713554987


  0%|          | 0/2344 [00:00<?, ?it/s]

epoch 2 batch id 1 loss 0.4531017243862152 train acc 0.84375
epoch 2 batch id 201 loss 0.18576008081436157 train acc 0.8776430348258707
epoch 2 batch id 401 loss 0.29717206954956055 train acc 0.8804551122194514
epoch 2 batch id 601 loss 0.4189072251319885 train acc 0.8837874376039934
epoch 2 batch id 801 loss 0.37122631072998047 train acc 0.8863334893882646


In [ ]:
# 테스트 문장 예측
test_sentence = '너무 좋아요!'
test_label = 0 # 실제 질병

unseen_test = pd.DataFrame([[test_sentence, test_label]], columns = [['사람문장1', '감정_대분류']])
unseen_values = unseen_test.values
test_set = BERTDataset(unseen_values, 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=5)

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  out = model(token_ids, valid_length, segment_ids)
  print(out)
    
    model.eval() # 평가 모드로 변경
    
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))